In [1]:
import pandas as pd
import pandera as pa
import numpy as np
import math
from pandasql import sqldf

#arquivo num googledrive
googleDriveZIP = '1jT_FREgV4_AhYJgRJ-iKzVYoRVarjgXW'
#googleDriveCSV = "1tp2J7vD9uwP8PcZM-Pzd1Zd_wny5K_ol" 
googleApiKey = 'x'

urlArquivo = f'https://www.googleapis.com/drive/v3/files/{googleDriveZIP}?alt=media&key={googleApiKey}'


In [2]:
#Monta DataFrame
df = pd.read_csv(urlArquivo, dtype=object, compression='zip')

In [3]:
df.describe()

,Name,Sex,Event,Equipment,Age,AgeClass,Division,BodyweightKg,WeightClassKg,Squat1Kg,...,McCulloch,Glossbrenner,IPFPoints,Tested,Country,Federation,Date,MeetCountry,MeetState,MeetName
count,1423354,1423354,1423354,1423354,757527,786800,1415176,1406622,1410042,337580,...,1304254,1304407,1273286,1093892,388884,1423354,1423354,1423354,941545,1423354
unique,412574,2,7,5,192,16,4842,11956,224,1343,...,59221,55340,65117,1,176,222,5367,96,111,11599
top,Alan Aerts,M,SBD,Single-ply,17.5,24-34,Open,90,90,200,...,94.16,106.12,491.92,Yes,USA,THSPA,2017-02-18,USA,TX,World Championships
freq,214,1060189,1073237,787141,22996,244197,337927,23343,103156,8153,...,175,225,183,1093892,91333,290547,7001,856561,448753,32615


In [4]:
#Novo DF só com info país = BR
filtroPais = df['Country'] == 'Brazil'
dfbrazil = df.loc[filtroPais]
dfbrazil.reset_index(drop=True, inplace=True)  # Drop evita indice antigo virar coluna
dfbrazil.head(2)

,Name,Sex,Event,Equipment,Age,AgeClass,Division,BodyweightKg,WeightClassKg,Squat1Kg,...,McCulloch,Glossbrenner,IPFPoints,Tested,Country,Federation,Date,MeetCountry,MeetState,MeetName
0,Marilia Coutinho,F,B,Raw,51,50-54,F-B,54.16,56,NaN,...,148.92,114.94,899.26,NaN,Brazil,RUPC,2015-02-07,USA,FL,Raw Unity VIII
1,Carlos Llosa,M,SBD,Raw,NaN,NaN,NaN,82.5,82.5,NaN,...,485.68,468.88,741.44,NaN,Brazil,RUPC,2016-01-31,USA,FL,Raw Unity IX


In [5]:
dfbrazil.columns.values

array(['Name', 'Sex', 'Event', 'Equipment', 'Age', 'AgeClass', 'Division',
       'BodyweightKg', 'WeightClassKg', 'Squat1Kg', 'Squat2Kg',
       'Squat3Kg', 'Squat4Kg', 'Best3SquatKg', 'Bench1Kg', 'Bench2Kg',
       'Bench3Kg', 'Bench4Kg', 'Best3BenchKg', 'Deadlift1Kg',
       'Deadlift2Kg', 'Deadlift3Kg', 'Deadlift4Kg', 'Best3DeadliftKg',
       'TotalKg', 'Place', 'Wilks', 'McCulloch', 'Glossbrenner',
       'IPFPoints', 'Tested', 'Country', 'Federation', 'Date',
       'MeetCountry', 'MeetState', 'MeetName'], dtype=object)

In [6]:
pd.options.mode.chained_assignment = None  # default='warn' desativa aviso de alteração em cópia do df original

#tem null
dfbrazil['Age'] = dfbrazil['Age'].astype('float64')
dfbrazil['Wilks'] = dfbrazil['Wilks'].astype('float64')


#Resultado preguiçoso GroupBy sem range idade
agewilks = dfbrazil.groupby(['Age'],dropna=True)['Wilks'].agg([len, min, max])
agewilks['len'] = agewilks['len'].astype('int')
agewilks.reset_index(inplace=True)
agewilks



,Age,len,min,max
0,13.0,1,168.78,168.78
1,13.5,5,79.78,305.52
2,14.0,5,116.10,173.89
3,14.5,19,49.37,369.69
4,15.0,16,74.35,373.13
...,...,...,...,...
114,74.5,1,314.52,314.52
115,75.5,2,325.28,332.17
116,76.5,1,275.26,275.26
117,77.5,1,292.33,292.33


In [7]:
#cópias teste
dfbrazil_copiateste = dfbrazil
dforiginal = dfbrazil

In [8]:
#groupby range 
melhorzinho = dfbrazil_copiateste.groupby(pd.cut(dfbrazil_copiateste["Age"],
                                                 np.arange(round(dfbrazil_copiateste["Age"].min(),-2), 
                                                           round(dfbrazil_copiateste["Age"].max(),-2),
                                                           10 #Incrementar certo resolveria tudo?
                                                          ),
                                                  right=True #não repete valor nos ranges
                                                 )
                                          )['Wilks'].agg([len, min, max])
melhorzinho                                        

,len,min,max
Age,,,
"(0.0, 10.0]",NaN,NaN,NaN
"(10.0, 20.0]",261.0,49.37,487.64
"(20.0, 30.0]",610.0,42.75,643.98
"(30.0, 40.0]",456.0,43.63,651.85
"(40.0, 50.0]",427.0,53.30,618.51
"(50.0, 60.0]",131.0,51.95,449.71
"(60.0, 70.0]",83.0,55.59,443.35
"(70.0, 80.0]",10.0,63.80,332.17
"(80.0, 90.0]",NaN,NaN,NaN


In [9]:
#teste 10-19
filtro_idade1 = dforiginal['Age'] <=20
t1 = dforiginal['Age'].loc[filtro_idade1 ].count()

#teste 20-29
filtro_idade11 = dforiginal['Age'] >20
filtro_idade22 = dforiginal['Age'] <=30
t2 = dforiginal['Age'].loc[filtro_idade11 & filtro_idade22 ].count()

#teste 50-59
filtro_idade11 = dforiginal['Age'] >50
filtro_idade22 = dforiginal['Age'] <=60
t3 = dforiginal['Age'].loc[filtro_idade11 & filtro_idade22 ].count()

print(f'Teste 10-19: {t1} | Teste 20-29: {t2} | Teste 50-59: {t3}')

Teste 10-19: 261 | Teste 20-29: 610 | Teste 50-59: 131


In [10]:
melhorzinho.reset_index(inplace=True)

In [11]:
#
def maqueiaAge(linha):
    linha[0] = linha[0][1:].strip()
    linha[1] = linha[1][:-1].strip()
    resultado = linha[0] + ' < Idade <= ' + linha[1]
    return resultado

melhorzinho['Age'] = melhorzinho['Age'].astype('string').str.split(',').apply(lambda x: maqueiaAge(x))

In [12]:
melhorzinho.dropna(inplace = True)
melhorzinho.rename(columns = {'Age':'Grupo Idade','len':'Qnt Atletas','min':'Wilks Min.','max':'Wilks Max.'}, inplace=True)
melhorzinho['Qnt Atletas'] = melhorzinho['Qnt Atletas'].astype('int64')
melhorzinho.dtypes

Grupo Idade     object
Qnt Atletas      int64
Wilks Min.     float64
Wilks Max.     float64
dtype: object

In [13]:
#final
schema = pa.DataFrameSchema({
    # built-in python types
    "Grupo Idade": pa.Column(str),
    "Qnt Atletas": pa.Column(int),
    "Wilks Min.": pa.Column(float),
    "Wilks Max.": pa.Column(float)
})

schema.validate(melhorzinho)

,Grupo Idade,Qnt Atletas,Wilks Min.,Wilks Max.
1,10.0 < Idade <= 20.0,261,49.37,487.64
2,20.0 < Idade <= 30.0,610,42.75,643.98
3,30.0 < Idade <= 40.0,456,43.63,651.85
4,40.0 < Idade <= 50.0,427,53.30,618.51
5,50.0 < Idade <= 60.0,131,51.95,449.71
6,60.0 < Idade <= 70.0,83,55.59,443.35
7,70.0 < Idade <= 80.0,10,63.80,332.17


In [14]:
#query pandaSQL
#Solução rápida hmm

testequery = """ WITH separa_grupos AS( SELECT Age, 
                        Wilks, 
                   CASE 
                        WHEN Age BETWEEN 10 AND 20 THEN '10.0 < Idade <= 20.0'
                        WHEN Age BETWEEN 20 AND 30 THEN '20.0 < Idade <= 30.0'
                        WHEN Age BETWEEN 30 AND 40 THEN '30.0 < Idade <= 40.0'
                        WHEN Age BETWEEN 40 AND 50 THEN '40.0 < Idade <= 50.0'
                        WHEN Age BETWEEN 50 AND 60 THEN '50.0 < Idade <= 60.0'
                        WHEN Age BETWEEN 60 AND 70 THEN '60.0 < Idade <= 70.0'
                        WHEN Age BETWEEN 70 AND 80 THEN '70.0 < Idade <= 80.0'

                    END Grupo
                   FROM dforiginal 
                  WHERE Country = "Brazil")

                  SELECT grupo AS 'Grupo Idade',
                         COUNT(*) AS 'Qnt Atletas',
                         MIN(Wilks) AS 'Wilks Min.',
                         MAX(Wilks) AS 'Wilks Max.' 
                    FROM separa_grupos
                   WHERE grupo IS NOT NULL
                GROUP BY GRUPO

             """
sqldf(testequery)

,Grupo Idade,Qnt Atletas,Wilks Min.,Wilks Max.
0,10.0 < Idade <= 20.0,261,49.37,487.64
1,20.0 < Idade <= 30.0,610,42.75,643.98
2,30.0 < Idade <= 40.0,456,43.63,651.85
3,40.0 < Idade <= 50.0,427,53.30,618.51
4,50.0 < Idade <= 60.0,131,51.95,449.71
5,60.0 < Idade <= 70.0,83,55.59,443.35
6,70.0 < Idade <= 80.0,10,63.80,332.17
